This is an [Apache `Spark`](https://spark.apache.org) variation of the [logistic scoring example](https://github.com/WinVector/data_algebra/blob/master/Examples/LogisticExample/ScoringExample.ipynb).

In [1]:
import pyspark
import pyspark.sql

import pandas

from data_algebra.data_ops import *
import data_algebra.SparkSQL

In [2]:
d_local = pandas.DataFrame({
    'subjectID':[1, 1, 2, 2],
    'surveyCategory': [ "withdrawal behavior", "positive re-framing", "withdrawal behavior", "positive re-framing"],
    'assessmentTotal': [5, 2, 3, 4],
    'irrelevantCol1': ['irrel1']*4,
    'irrelevantCol2': ['irrel2']*4,
})
d_local

,subjectID,surveyCategory,assessmentTotal,irrelevantCol1,irrelevantCol2
0,1,withdrawal behavior,5,irrel1,irrel2
1,1,positive re-framing,2,irrel1,irrel2
2,2,withdrawal behavior,3,irrel1,irrel2
3,2,positive re-framing,4,irrel1,irrel2


In [3]:
sc = pyspark.SparkContext()
spark = pyspark.sql.SparkSession.builder.appName('pandasToSparkDF').getOrCreate()

d_spark = spark.createDataFrame(d_local)
d_spark

DataFrame[subjectID: bigint, surveyCategory: string, assessmentTotal: bigint, irrelevantCol1: string, irrelevantCol2: string]

In [4]:
d_spark.createOrReplaceTempView("d")
sql_df = spark.sql("SELECT * FROM d")
sql_df.show()

+---------+-------------------+---------------+--------------+--------------+
|subjectID|     surveyCategory|assessmentTotal|irrelevantCol1|irrelevantCol2|
+---------+-------------------+---------------+--------------+--------------+
|        1|withdrawal behavior|              5|        irrel1|        irrel2|
|        1|positive re-framing|              2|        irrel1|        irrel2|
|        2|withdrawal behavior|              3|        irrel1|        irrel2|
|        2|positive re-framing|              4|        irrel1|        irrel2|
+---------+-------------------+---------------+--------------+--------------+



In [5]:
local_copy = pandas.DataFrame(sql_df.collect())
local_copy

,0,1,2,3,4
0,1,withdrawal behavior,5,irrel1,irrel2
1,1,positive re-framing,2,irrel1,irrel2
2,2,withdrawal behavior,3,irrel1,irrel2
3,2,positive re-framing,4,irrel1,irrel2


In [6]:
scale = 0.237

ops = data_algebra.data_ops.describe_table(d_local, 'd'). \
    extend({'probability': f'(assessmentTotal * {scale}).exp()'}). \
    extend({'total': 'probability.sum()'},
           partition_by='subjectID'). \
    extend({'probability': 'probability/total'}). \
    extend({'sort_key': '-probability'}). \
    extend({'row_number': '_row_number()'},
           partition_by=['subjectID'],
           order_by=['sort_key']). \
    select_rows('row_number == 1'). \
    select_columns(['subjectID', 'surveyCategory', 'probability']). \
    rename_columns({'diagnosis': 'surveyCategory'})
    
print(ops.to_python(pretty=True))

TableDescription(
    table_name="d",
    column_names=[
        "subjectID",
        "surveyCategory",
        "assessmentTotal",
        "irrelevantCol1",
        "irrelevantCol2",
    ],
).extend({"probability": "(assessmentTotal * 0.237).exp()"}).extend(
    {"total": "probability.sum()"}, partition_by=["subjectID"]
).extend(
    {"probability": "probability / total"}
).extend(
    {"sort_key": "-probability"}
).extend(
    {"row_number": "_row_number()"}, partition_by=["subjectID"], order_by=["sort_key"]
).select_rows(
    "row_number == 1"
).select_columns(
    ["subjectID", "surveyCategory", "probability"]
).rename_columns(
    {"diagnosis": "surveyCategory"}
)



In [7]:
db_model = data_algebra.SparkSQL.SparkSQLModel()
sql = ops.to_sql(db_model, pretty=True)
print(sql)

SELECT `probability`,
       `subjectid`,
       `surveycategory` AS `diagnosis`
FROM (
SELECT `probability`,
       `surveycategory`,
       `subjectid`
FROM (
SELECT `probability`,
       `surveycategory`,
       `subjectid`
FROM (
SELECT `probability`,
       `surveycategory`,
       `sort_key`,
       `subjectid`,
       ROW_NUMBER() OVER (PARTITION BY `subjectid`
                          ORDER BY `sort_key`) AS `row_number`
FROM (
SELECT `probability`,
       `surveycategory`,
       `subjectid`, ( -`probability` ) AS `sort_key` FROM ( SELECT `surveycategory`, `subjectid`, `probability` / `total` AS `probability` FROM ( SELECT `probability`, `surveycategory`, `subjectid`, SUM(`probability`) OVER ( PARTITION BY `subjectid`  )  AS `total` FROM ( SELECT `surveycategory`, `subjectid`, EXP((`assessmenttotal` * 0.237)) AS `probability` FROM ( SELECT `assessmenttotal`, `surveycategory`, `subjectid` FROM `d` ) `sq_0` ) `sq_1` ) `sq_2` ) `sq_3` ) `sq_4` ) `sq_5` WHERE `row_number` = 1 ) `

In [8]:
sql_q = spark.sql(sql)


In [9]:
res = pandas.DataFrame(sql_q.collect())
res

,0,1,2
0,0.670622,1,withdrawal behavior
1,0.558974,2,positive re-framing


In [10]:
sc.stop()